In [1]:
import time
import pickle
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

Matplotlib is building the font cache; this may take a moment.


In [2]:
# path = "Analytics/Personal Folders/liuwei01/2023/JPMC/new_data_run_0316_prod/"
path = "Analytics/Personal Folders/liuwei01/2023/JPMC/new_data_run_0319_prod/"

In [3]:
file1 = pd.read_adls(path + "z_LN_TrendV_Bal_with_sv_0321_prod_v3.csv")
file2 = pd.read_adls(path + 'z_LN_TrendV_Util_with_sv_0321_prod_v3.csv')
file3 = pd.read_adls(path + 'z_ln_core_with_sv_0321_prod_v3.csv')

print(file1.shape)
print(file2.shape)
print(file3.shape)

(34550, 142)
(34550, 114)
(34550, 27)


In [4]:
merged = file1.copy()
merged =merged.merge(file2, on = "scramble_2")
merged =merged.merge(file3, on = "scramble_2")
print(merged.shape)

(34550, 281)


In [5]:
file3.columns

Index(['scramble_2', 'snapshot_dt', 'BS001_num_fin_accts',
       'BS002_num_acct_rev_dlq_1p_12m', 'BS003_num_acct_inst_dlq_2p_12m',
       'BS004_past_due_amt_loan', 'BS005_past_due_amt_loc',
       'BS006_past_due_amt_leas', 'B07_max_bal', 'BS007_tot_cred_rem_max',
       'BS008_share_open_not_dlq', 'BS009_num_cards_pos_bal',
       'BS010_current_util_card_nocf', 'BS011_util_fin_12m_nocf',
       'BS012_util_rev_3m_nocf', 'BS013_wrst_bckt_max_12m_fin',
       'BS014_wrst_bckt_max_3m_fin', 'BS015_wrst_bckt_max_1m_fin',
       'BS016_months_since_open_max', 'BS017_cur_rem_bal_card',
       'BS018_cur_rem_bal_loan', 'BS019_cur_rem_bal_loc',
       'BS020_sum_vel_avg_11m_rev', 'BS021_cur_rem_bal_rev',
       'BS022_total_cur_cred_lmt_rev', 'BS023_current_util_rev_nocf',
       'BS024_num_open_accts_pos_bal'],
      dtype='object')

In [6]:
merged.columns = [x.lower() for x in merged.columns]

In [7]:
keep = ['scramble_2', 'snapshot_dt', 'bs035_bal_m0_fin', 'bs036_bal_m0_card', 'bs033_bal_min_3m_rev', 'bs034_bal_min_3m_card', 'bs066_bal_avg_12m_rev', 'bs045_bal_dif_m0_m11_fin', 
'bs043_bal_dif_m0_m11_rev', 'bs046_bal_dif_m0_m11_card', 'bs044_bal_dif_m0_m11_loc', 'bs050_bal_dif_cur_12mavg_fin', 'bs049_bal_dif_cur_12mmax_rev', 'bs048_bal_dif_cur_6mmax_fin', 
'bs047_bal_dif_cur_6mmax_rev', 'bs064_bal_ch_abs_max_12m_rev', 'bs062_bal_ch_abs_min_12m_rev', 'bs065_bal_ch_abs_max_12m_card', 'bs063_bal_ch_abs_min_12m_card', 
'bs058_bal_ch_min_12m_fin', 'bs057_bal_ch_min_12m_rev', 'bs056_bal_ch_min_3m_fin', 'bs060_bal_ch_avg_12m_fin', 'bs059_bal_ch_avg_12m_rev', 'bs061_bal_ch_avg_12m_card', 
'bs055_bal_ch_neg_sum_6m_fin', 'bs054_bal_ch_neg_sum_6m_rev', 'bs031_bal_rto_cur_12mmax_fin', 'bs030_bal_rto_cur_12mmax_rev', 'bs032_bal_rto_cur_12mmax_card', 
'bs028_bal_rto_cur_6mmax_fin', 'bs027_bal_rto_cur_6mmax_rev', 'bs029_bal_rto_cur_6mmax_card', 'bs040_bal_dif_r_m0_m11_rev', 'bs042_bal_dif_r_m0_m11_card', 
'bs041_bal_dif_r_m0_m11_loc', 'bs052_bal_ch_r_min_6m_fin', 'bs051_bal_ch_r_min_6m_rev', 'bs053_bal_ch_r_min_6m_card', 'bs039_bal_inc_cnt_12m_fin', 'bs038_bal_inc_cnt_6m_card', 
'bs037_bal_inc_cnt_6m_rev', 'bs026_bal_trend_ind_3m_fin', 'bs025_bal_trend_ind_3m_rev', 'bs078_util_m0_loc', 'bs076_util_max_3m_rev', 'bs077_util_max_12m_card', 
'bs074_util_max_6m_rev', 'bs075_util_max_6m_card', 'bs073_util_min_3m_rev', 'bs071_util_min_6m_rev', 'bs072_util_min_6m_card', 'bs087_util_avg_6m_rev', 'bs089_util_avg_3m_card', 
'bs088_util_avg_3m_loc', 'bs081_util_dif_m0_m11_rev', 'bs082_util_dif_m0_m11_card', 'bs085_util_ch_avg_6m_rev', 'bs086_util_ch_avg_12m_card', 'bs079_util_inc_cnt_12m_rev', 
'bs080_util_inc_cnt_12m_card', 'bs092_util_75_cnt_12m_open', 'bs091_util_75_cnt_12m_rev', 'bs093_util_75_cnt_12m_card', 'bs090_util_75_cnt_3m_rev', 'bs070_util_rto_cur_12mmax_card', 
'bs069_util_rto_cur_12mmax_rev', 'bs083_util_ch_r_min_6m_rev', 'bs084_util_ch_r_min_6m_card', 'bs067_util_trend_ind_3m_rev', 'bs068_util_trend_ind_3m_card', 'bs007_tot_cred_rem_max', 
'bs022_total_cur_cred_lmt_rev', 'bs023_current_util_rev_nocf', 'bs010_current_util_card_nocf', 'bs011_util_fin_12m_nocf', 'bs012_util_rev_3m_nocf', 'bs016_months_since_open_max', 
'bs001_num_fin_accts', 'bs024_num_open_accts_pos_bal', 'bs009_num_cards_pos_bal', 'bs021_cur_rem_bal_rev', 'bs017_cur_rem_bal_card', 'bs019_cur_rem_bal_loc', 
'bs018_cur_rem_bal_loan', 'bs020_sum_vel_avg_47m_rev', 'bs020_sum_vel_avg_11m_rev', 'bs002_num_acct_rev_dlq_1p_48m', 'bs002_num_acct_rev_dlq_1p_12m', 
'bs003_num_acct_inst_dlq_2p_48m', 'bs003_num_acct_inst_dlq_2p_12m', 'bs015_wrst_bckt_max_1m_fin', 'bs013_wrst_bckt_max_48m_fin', 'bs013_wrst_bckt_max_12m_fin', 
'bs014_wrst_bckt_max_3m_fin', 'bs008_share_open_not_dlq', 'bs004_past_due_amt_loan', 'bs005_past_due_amt_loc', 'bs006_past_due_amt_leas', ]
print(len(keep))

99


In [8]:
[x for x in keep if x not in merged.columns]

['bs020_sum_vel_avg_47m_rev',
 'bs002_num_acct_rev_dlq_1p_48m',
 'bs003_num_acct_inst_dlq_2p_48m',
 'bs013_wrst_bckt_max_48m_fin']

In [9]:
temp = merged[[x for x in keep if x in merged.columns]].copy()

In [10]:
temp = temp.rename(columns = {"scramble_2":"account", 'snapshot_dt': "history_date"})

In [20]:
temp.to_adls(path + "file_for_heberton_prod_0321_run.csv", format = ".csv", index = False, overwrite = True)

Bad pipe message: %s [b'X\x08\xc4\xbe\xdd\xae#\xc0\xf5\xbd\xcc[\xd3\xf5\x8b\xe6=\xa8 E\xb2\xf1\xbd\x1e1\xb8\xcb(\xcd\xb5#oWs\x16\x82\xbd\x1c5\xaf\xb2,\xd2\x80D\xd9i\xf4\xda|Y\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04']
Bad pipe message: %s [b'', b'']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 L\x10OL\\\x07\xe6\x9d\x8c\xf3\xf3c{|N\xdd\x81\x86\xc6\xaa\x02\x9f']
Bad pipe message: %s [b'\xf7\xdd:\xf4\x88\xac\x16h\x98\xaa\x0c@Vn\x1f\x92\xcd \x00\x00>\xc0\x14\xc0\n\x009\x008\x007\x006\xc0\x0f\xc0\x05\x005\xc0\x13\xc0\t\x003\x002\x001\x000\xc0\x0e\xc0\x04\x00/\x00\x9a\x00\x99']
Bad pipe message: %s [b'@fFg%(sQi\x16\xe7\x17

## ECL output

In [13]:
ecl = pd.read_adls(path + "0321_armbs_1803_deliverables_attributes_50k_roxie_w20230321-165050.csv")
print(ecl.shape)

(51298, 105)


In [14]:
temp.columns = ["sas_" + x for x in temp.columns]
merged = temp.merge(ecl, left_on = "sas_account", right_on = "account", how = 'outer')
print(merged.shape)

(51298, 200)


In [15]:
merged = merged[merged.sas_account.isnull()]
print(merged.shape)

(16748, 200)


In [19]:
pd.unique(merged[['bs035_bal_m0_fin', 'bs036_bal_m0_card', 'bs033_bal_min_3m_rev', 'bs034_bal_min_3m_card', 'bs066_bal_avg_12m_rev', 'bs045_bal_dif_m0_m11_fin', 
'bs043_bal_dif_m0_m11_rev', 'bs046_bal_dif_m0_m11_card', 'bs044_bal_dif_m0_m11_loc', 'bs050_bal_dif_cur_12mavg_fin', 'bs049_bal_dif_cur_12mmax_rev', 'bs048_bal_dif_cur_6mmax_fin', 
'bs047_bal_dif_cur_6mmax_rev', 'bs064_bal_ch_abs_max_12m_rev', 'bs062_bal_ch_abs_min_12m_rev', 'bs065_bal_ch_abs_max_12m_card', 'bs063_bal_ch_abs_min_12m_card', 
'bs058_bal_ch_min_12m_fin', 'bs057_bal_ch_min_12m_rev', 'bs056_bal_ch_min_3m_fin', 'bs060_bal_ch_avg_12m_fin', 'bs059_bal_ch_avg_12m_rev', 'bs061_bal_ch_avg_12m_card', 
'bs055_bal_ch_neg_sum_6m_fin', 'bs054_bal_ch_neg_sum_6m_rev', 'bs031_bal_rto_cur_12mmax_fin', 'bs030_bal_rto_cur_12mmax_rev', 'bs032_bal_rto_cur_12mmax_card', 
'bs028_bal_rto_cur_6mmax_fin', 'bs027_bal_rto_cur_6mmax_rev', 'bs029_bal_rto_cur_6mmax_card', 'bs040_bal_dif_r_m0_m11_rev', 'bs042_bal_dif_r_m0_m11_card', 
'bs041_bal_dif_r_m0_m11_loc', 'bs052_bal_ch_r_min_6m_fin', 'bs051_bal_ch_r_min_6m_rev', 'bs053_bal_ch_r_min_6m_card', 'bs039_bal_inc_cnt_12m_fin', 'bs038_bal_inc_cnt_6m_card', 
'bs037_bal_inc_cnt_6m_rev', 'bs026_bal_trend_ind_3m_fin', 'bs025_bal_trend_ind_3m_rev', 'bs078_util_m0_loc', 'bs076_util_max_3m_rev', 'bs077_util_max_12m_card', 
'bs074_util_max_6m_rev', 'bs075_util_max_6m_card', 'bs073_util_min_3m_rev', 'bs071_util_min_6m_rev', 'bs072_util_min_6m_card', 'bs087_util_avg_6m_rev', 'bs089_util_avg_3m_card', 
'bs088_util_avg_3m_loc', 'bs081_util_dif_m0_m11_rev', 'bs082_util_dif_m0_m11_card', 'bs085_util_ch_avg_6m_rev', 'bs086_util_ch_avg_12m_card', 'bs079_util_inc_cnt_12m_rev', 
'bs080_util_inc_cnt_12m_card', 'bs092_util_75_cnt_12m_open', 'bs091_util_75_cnt_12m_rev', 'bs093_util_75_cnt_12m_card', 'bs090_util_75_cnt_3m_rev', 'bs070_util_rto_cur_12mmax_card', 
'bs069_util_rto_cur_12mmax_rev', 'bs083_util_ch_r_min_6m_rev', 'bs084_util_ch_r_min_6m_card', 'bs067_util_trend_ind_3m_rev', 'bs068_util_trend_ind_3m_card', 'bs007_tot_cred_rem_max', 
'bs022_total_cur_cred_lmt_rev', 'bs023_current_util_rev_nocf', 'bs010_current_util_card_nocf', 'bs011_util_fin_12m_nocf', 'bs012_util_rev_3m_nocf', 'bs016_months_since_open_max', 
'bs001_num_fin_accts', 'bs024_num_open_accts_pos_bal', 'bs009_num_cards_pos_bal', 'bs021_cur_rem_bal_rev', 'bs017_cur_rem_bal_card', 'bs019_cur_rem_bal_loc', 
'bs018_cur_rem_bal_loan', 'bs020_sum_vel_avg_47m_rev', 'bs020_sum_vel_avg_11m_rev', 'bs002_num_acct_rev_dlq_1p_48m', 'bs002_num_acct_rev_dlq_1p_12m', 
'bs003_num_acct_inst_dlq_2p_48m', 'bs003_num_acct_inst_dlq_2p_12m', 'bs015_wrst_bckt_max_1m_fin', 'bs013_wrst_bckt_max_48m_fin', 'bs013_wrst_bckt_max_12m_fin', 
'bs014_wrst_bckt_max_3m_fin', 'bs008_share_open_not_dlq', 'bs004_past_due_amt_loan', 'bs005_past_due_amt_loc', 'bs006_past_due_amt_leas', ]].values.ravel('K'))

array([999999999, 999999990,       999,       990])

In [13]:
ecl.columns

Index(['account', 'id_powid', 'id_proxid', 'id_seleid', 'id_orgid', 'id_ultid',
       'history_date', 'history_datetime', 'bs035_bal_m0_fin',
       'bs036_bal_m0_card',
       ...
       'bs003_num_acct_inst_dlq_2p_48m', 'bs003_num_acct_inst_dlq_2p_12m',
       'bs015_wrst_bckt_max_1m_fin', 'bs013_wrst_bckt_max_48m_fin',
       'bs013_wrst_bckt_max_12m_fin', 'bs014_wrst_bckt_max_3m_fin',
       'bs008_share_open_not_dlq', 'bs004_past_due_amt_loan',
       'bs005_past_due_amt_loc', 'bs006_past_due_amt_leas'],
      dtype='object', length=105)

In [14]:
from dsgtools.reporting import ImpactAnalysisReport
cr = ImpactAnalysisReport(temp, ecl, keys="account", dataframe_names =("sas", "ecl"))
cr.to_xlsx("./_temp/sas_vs_ecl_jpmc.xlsx", overwrite=True)

In [15]:
temp.columns = ["sas_" + x for x in temp.columns]
merged = temp.merge(ecl, left_on = "sas_account", right_on = "account")
print(merged.shape)

(34550, 200)


In [16]:
pd.set_option('display.float_format', lambda x: '%.f' % x)
merged.loc[merged["bs001_num_fin_accts"]!= merged["sas_bs001_num_fin_accts"], ["account", 'id_powid', 'id_proxid', 'id_seleid', 'id_orgid', 'id_ultid',
       'history_date', 'history_datetime', "sas_bs001_num_fin_accts", "bs001_num_fin_accts"]]

,account,id_powid,id_proxid,id_seleid,id_orgid,id_ultid,history_date,history_datetime,sas_bs001_num_fin_accts,bs001_num_fin_accts
7286,q177p4on22q,NaN,NaN,NaN,NaN,NaN,0,0,1,NaN
7878,q177p4p7816,NaN,NaN,NaN,NaN,NaN,0,0,2,NaN
9962,q177p7171n7,NaN,NaN,NaN,NaN,NaN,0,0,2,NaN


In [18]:
pd.set_option('display.float_format', lambda x: '%.f' % x)
merged.loc[merged["bs006_past_due_amt_leas"]!= merged["sas_bs006_past_due_amt_leas"], ["account", 'id_powid', 'id_proxid', 'id_seleid', 'id_orgid', 'id_ultid',
       'history_date', 'history_datetime', "bs001_num_fin_accts", "sas_bs006_past_due_amt_leas", "bs006_past_due_amt_leas"]]

,account,id_powid,id_proxid,id_seleid,id_orgid,id_ultid,history_date,history_datetime,bs001_num_fin_accts,sas_bs006_past_due_amt_leas,bs006_past_due_amt_leas
1134,q177p1n67p8,136796291670,136796291670,160607872,160607872,160607872,202103,20210331,6,999999996,999999997
5285,q177p47p22p,157358768,157358768,157358768,157358768,157358768,202103,20210331,5,999999996,999999997
5309,q177p47pnq4,157358768,157358768,157358768,157358768,157358768,202103,20210331,5,999999996,999999997
7286,q177p4on22q,NaN,NaN,NaN,NaN,NaN,0,0,NaN,999999991,NaN
7878,q177p4p7816,NaN,NaN,NaN,NaN,NaN,0,0,NaN,999999991,NaN
8177,q177p4pqqpq,80033702,80033702,80033702,80033702,80033702,202103,20210331,9,999999996,999999997
8315,q177p4q447q,136207511894,136207511894,68561472,68561472,68561472,202103,20210331,4,999999996,999999997
9695,q177p6pn4q4,134444141566,134444141566,142406685,142406685,142406685,202012,20201231,3,999999996,999999997
9962,q177p7171n7,NaN,NaN,NaN,NaN,NaN,0,0,NaN,999999991,NaN
9971,q177p717486,202759238,202759238,41949543,41949543,41949543,202012,20201231,1,999999996,999999997


## Compare to the prior run

In [19]:
path = "Analytics/Personal Folders/liuwei01/2023/JPMC/new_data_run_0316_prod/"
file1 = pd.read_adls(path + "z_LN_TrendV_Bal_with_sv_0315_prod_v1.csv")
file2 = pd.read_adls(path + 'z_LN_TrendV_Util_with_sv_0315_prod_v1.csv')
file3 = pd.read_adls(path + 'z_ln_core_with_sv_0315_prod_v1.csv')
merged = file1.copy()
merged =merged.merge(file2, on = "scramble_2")
merged =merged.merge(file3, on = "scramble_2")
print(merged.shape)
merged.columns = [x.lower() for x in merged.columns]
temp_old = merged[[x for x in keep if x in merged.columns]].copy()
temp_old = temp_old.rename(columns = {"scramble_2":"account", 'snapshot_dt': "history_date"})

(34550, 281)


In [20]:
cr = ImpactAnalysisReport(temp, temp_old, keys="account", dataframe_names =("sas", "sas_old"))
cr.to_xlsx("./_temp/sas_vs_sas_old_jpmc.xlsx", overwrite=True)